Before running this code, you need to set up the Earth Engine Python API.
Follow these steps (based on official Google documentation):
1. Install the Earth Engine API if not already installed (e.g., in a terminal: pip install earthengine-api --upgrade).
2. Create a Google Cloud Project: Go to [link](https://console.cloud.google.com/projectcreate) and create a new project. Note your Project ID (e.g., 'my-earth-engine-project').
3. Enable the Earth Engine API: In your Google Cloud Console, search for "Earth Engine API" and enable it for your project.
4. Authenticate: Run ee.Authenticate() below if prompted; this will open a browser for Google sign-in and generate a token.
5. Initialize: Replace 'your-project-id' below with your actual Google Cloud Project ID.
For more details, see: [link](https://developers.google.com/earth-engine/guides/python_install)

# Initialization

In [1]:
#import relevant libraries
import ee
import geemap

In [2]:
#Authenticate and initilaize GEE 
try:
    ee.Initialize(project='uss-dissertation')
except Exception:
    print("GEE initialization failed. Run ee.Authenticate() and try again.")
    ee.Authenticate()
    ee.Initialize(project='uss-dissertation')

# Calculations

Normalized Difference Vegetation Index (NDVI) and Normalized Difference Impervious Surface Index (NDISI) are both considered as potential factors to be assessed.

Equations used as per the following:([Xu,2010](https://www.researchgate.net/publication/232724051_Analysis_of_Impervious_Surface_and_its_Impact_on_Urban_Heat_Environment_using_the_Normalized_Difference_Impervious_Surface_Index_NDISI) ; [Li et al., 2020](https://www.mdpi.com/2073-445X/9/2/57); [Peng et al., 2021](https://www-sciencedirect-com.libproxy.ucl.ac.uk/science/article/pii/S0034425720305083#s0010))

 $$NDVI = \frac{NIR - Red}{NIR + Red}$$
  
  Where:
  - NIR is the near-infrared band.
  - Red is the red band.

 $$NDISI = \frac{TIR - (VIS1 + NIR + MIR1)/3}{TIR + (VIS1 + NIR + MIR1)/3}$$
  
  Where:
  - TIR is the thermal infrared band in Kelvin (normalized to 0-1).
  - VIS1 is a visible band (green, SR_B3) to suppress water noise as per Xu (2010).
  - NIR is the near-infrared band (SR_B5, normalized).
  - MIR1 is the mid-infrared band (SWIR1, SR_B6, normalized).

* Note: I'm using Landsat for consistency across layers at 30m resolution instead of Sentinal-2 10m resolution for NDVI.

In [3]:
# Load city boundary from a Google Earth Engine asset.
def load_city_boundary(asset_id):
    try:
        city_boundary = ee.FeatureCollection(asset_id)
        print("City boundary loaded successfully!")
        return city_boundary
    except Exception as e:
        print(f"Failed to load city boundary: {str(e)}")
        return None

# Calculate NDVI and NDISI for 2024 from Landsat for the specified year (consistency at 30m resolution).
def calculate_ndvi_ndisi(city_boundary, years):
    ndvi_layers = {}
    ndisi_layers = {}
    geometry = city_boundary.geometry()
    for year in years: 
        # Filter for summer months to align with LST data
        start_date = f'{year}-06-01'
        end_date = f'{year}-08-31'

        landsat = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                   .filterBounds(geometry)
                   .filterDate(start_date, end_date)
                   .filter(ee.Filter.lt('CLOUD_COVER', 5)))# cloud cover (<5%) to minimize atmospheric interference.
        landsat = landsat.merge(
            ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
            .filterBounds(geometry)
            .filterDate(start_date, end_date)
            .filter(ee.Filter.lt('CLOUD_COVER', 5))
        )
        landsat_size = landsat.size().getInfo()
        print(f"Number of Landsat images for {year}: {landsat_size}")
        if landsat_size > 0:
            image_list = landsat.toList(landsat.size())
            for i in range(landsat_size):
                image = ee.Image(image_list.get(i))
                spacecraft = image.get('SPACECRAFT_ID').getInfo()
                path = image.get('WRS_PATH').getInfo()
                row = image.get('WRS_ROW').getInfo()
                date_acquired = image.get('DATE_ACQUIRED').getInfo()
                image_id = image.get('system:id').getInfo()
                print(f"Image {i+1}: {spacecraft} - Path: {path} - Row: {row} - Date: {date_acquired} - ID: {image_id}")
            
            # Compute mean composite for average summer conditions.
            landsat = landsat.mean()  
            # Compute surface reflectance (divide by 10000 as per Landsat scaling to get 0-1 range).
            reflectance = landsat.divide(10000)
            
            # NDVI formula: (NIR - Red) / (NIR + Red), using SR_B5 (NIR) and SR_B4 (Red).
            ndvi = reflectance.normalizedDifference(['SR_B5', 'SR_B4']).rename(f'NDVI_{year}').clip(geometry).unmask(0)
            ndvi_layers[year] = ndvi
            print(f"NDVI for {year} calculated successfully!")
            
            # VIS1: Green band (SR_B3) as per paper's band 2 equivalent for water suppression.
            vis1 = reflectance.select('SR_B3').rename('VIS1')
            nir = reflectance.select('SR_B5').rename('NIR')  # NIR: SR_B5
            mir1 = reflectance.select('SR_B6').rename('MIR1')  # MIR1: SR_B6 (SWIR1)
            # TIR (Thermal Infrared) in Kelvin: ST_B10 * scale + offset.
            tir = landsat.select('ST_B10').multiply(0.00341802).add(149).rename('TIR')

            # Combine for min-max stats to normalize (ensures bands are comparable, preventing TIR dominance).
            combined = tir.addBands(vis1).addBands(nir).addBands(mir1)
            stats = combined.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=geometry,
                scale=30,
                maxPixels=1e9
            ).getInfo()
            
            # Normalize each to 0-1 (addresses scale mismatch; Kelvin vs. reflectance).
            tir_norm = tir.subtract(stats['TIR_min']).divide(stats['TIR_max'] - stats['TIR_min'])
            vis1_norm = vis1.subtract(stats['VIS1_min']).divide(stats['VIS1_max'] - stats['VIS1_min'])
            nir_norm = nir.subtract(stats['NIR_min']).divide(stats['NIR_max'] - stats['NIR_min'])
            mir1_norm = mir1.subtract(stats['MIR1_min']).divide(stats['MIR1_max'] - stats['MIR1_min'])
            
            # Compute average of (VIS1_norm + NIR_norm + MIR1_norm) / 3
            avg = vis1_norm.add(nir_norm).add(mir1_norm).divide(3)
            # NDISI formula: (TIR_norm - Avg) / (TIR_norm + Avg)
            # Clipped to geometry and unmasked with 0 for no-data areas.
            ndisi = tir_norm.subtract(avg).divide(tir_norm.add(avg)).clip(geometry).unmask(0).rename(f'NDISI_{year}')
            ndisi_layers[year] = ndisi
            print(f"NDISI for {year} calculated successfully!")
        else:
            print(f"No valid Landsat data for {year}")
            ndvi_layers[year] = None
            ndisi_layers[year] = None
    return ndvi_layers, ndisi_layers

# Add NDVI and NDISI layers to the map
def add_layers(map_obj, city_boundary, ndvi_layers, ndisi_layers):
    """
    Adds NDVI and NDISI layers to the geemap Map object with dynamic visualization parameters.
    """
    geometry = city_boundary.geometry()
    # NDVI visualization: Blue (low vegetation) to green (high vegetation).
    ndvi_palette = ['blue', 'white', 'green']
    ndvi_vis = {"min": -1, "max": 1, "palette": ndvi_palette}
    
    # NDISI visualization: Green (pervious/vegetated) to brown (impervious surfaces).
    ndisi_palette = ['green', 'white', 'brown']
    ndisi_vis = {"min": -1, "max": 1, "palette": ndisi_palette}
    
    for year in years:
        if ndvi_layers.get(year) is not None:
            map_obj.addLayer(ndvi_layers[year], ndvi_vis, f'NDVI_{year}')
        if ndisi_layers.get(year) is not None:
            map_obj.addLayer(ndisi_layers[year], ndisi_vis, f'NDISI_{year}')
    
    return map_obj

# Export layers as GeoTIFFs to Google Drive
def export_layers(ndvi_layers, ndisi_layers, city_boundary):
    """
    Exports NDVI and NDISI rasters to Google Drive as GeoTIFFs for further analysis.
    """
    geometry = city_boundary.geometry()
    for year in years:
        if ndvi_layers.get(year) is not None:
            ee.batch.Export.image.toDrive(
                image=ndvi_layers[year],
                description=f'Riyadh_NDVI_EPSG20438_{year}',
                folder='Riyadh_Layers',
                fileNamePrefix=f'Riyadh_NDVI_EPSG20438_{year}',
                region=geometry,
                scale=30,  # Landsat resolution for NDVI
                crs='EPSG:20438', # UTM Zone 38N for metric accuracy as per https://epsg.io/20438
                maxPixels=1e9
            ).start()
            print(f"Exporting NDVI_{year} to Google Drive")
        if ndisi_layers.get(year) is not None:
            ee.batch.Export.image.toDrive(
                image=ndisi_layers[year],
                description=f'Riyadh_NDISI_EPSG20438_{year}',
                folder='Riyadh_Layers',
                fileNamePrefix=f'Riyadh_NDISI_EPSG20438_{year}',
                region=geometry,
                scale=30,  # Landsat resolution for NDISI
                crs='EPSG:20438', # UTM Zone 38N for metric accuracy as per https://epsg.io/20438
                maxPixels=1e9
            ).start()
            print(f"Exporting NDISI_{year} to Google Drive")

# Load and map
asset_id = 'projects/uss-dissertation/assets/riyadh_boundary'  #link to asset: https://code.earthengine.google.com/?asset=projects/uss-dissertation/assets/riyadh_boundary
city_boundary = load_city_boundary(asset_id)
map_obj = geemap.Map(basemap='CartoDB.Positron')
boundary_style = {"color": "black", "fillColor": "00000000", "width": 2}
map_obj.addLayer(city_boundary, boundary_style, "Riyadh City Boundary")
map_obj.setCenter(46.75, 24.78, 12)

# Calculate and add layers
years = [2024]
ndvi_layers, ndisi_layers = calculate_ndvi_ndisi(city_boundary, years)
map_obj = add_layers(map_obj, city_boundary, ndvi_layers, ndisi_layers)

# Comment export for now
export_layers(ndvi_layers, ndisi_layers, city_boundary)

map_obj

City boundary loaded successfully!
Number of Landsat images for 2024: 32
Image 1: LANDSAT_8 - Path: 165 - Row: 43 - Date: 2024-06-01 - ID: LANDSAT/LC08/C02/T1_L2/LC08_165043_20240601
Image 2: LANDSAT_8 - Path: 165 - Row: 43 - Date: 2024-06-17 - ID: LANDSAT/LC08/C02/T1_L2/LC08_165043_20240617
Image 3: LANDSAT_8 - Path: 165 - Row: 43 - Date: 2024-07-03 - ID: LANDSAT/LC08/C02/T1_L2/LC08_165043_20240703
Image 4: LANDSAT_8 - Path: 165 - Row: 43 - Date: 2024-07-19 - ID: LANDSAT/LC08/C02/T1_L2/LC08_165043_20240719
Image 5: LANDSAT_8 - Path: 165 - Row: 43 - Date: 2024-08-04 - ID: LANDSAT/LC08/C02/T1_L2/LC08_165043_20240804
Image 6: LANDSAT_8 - Path: 165 - Row: 43 - Date: 2024-08-20 - ID: LANDSAT/LC08/C02/T1_L2/LC08_165043_20240820
Image 7: LANDSAT_8 - Path: 166 - Row: 42 - Date: 2024-06-08 - ID: LANDSAT/LC08/C02/T1_L2/LC08_166042_20240608
Image 8: LANDSAT_8 - Path: 166 - Row: 42 - Date: 2024-06-24 - ID: LANDSAT/LC08/C02/T1_L2/LC08_166042_20240624
Image 9: LANDSAT_8 - Path: 166 - Row: 42 - Date

Map(center=[24.78, 46.75], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…